In [1]:
# Import dependencies
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score

In [2]:
# Import our input dataset
tele_df = pd.read_csv('bank_telemarketing.csv')
tele_df.head()

,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


In [3]:
# Generate our categorial variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == 'object'].index.tolist()

# Check number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

In [4]:
# Create OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform 
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [5]:
# Merge to original dataframe
tele_df = tele_df.merge(encode_df, left_index=True, right_index=True)
tele_df = tele_df.drop(tele_cat, axis=1)
tele_df.head()

,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [6]:
# Remove loan status target
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns=['Subscribed_no', 'Subscribed_yes'], axis=1)

# Split training/testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler
scaler = StandardScaler()

# Fit Scaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# SVM

# Create SVM model
svm = SVC(kernel='linear')

# Train model
svm.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = svm.predict(X_test_scaled)
print(f'SMV model accuracy: {accuracy_score(y_test, y_pred):.3f}')

SMV model accuracy: 0.876


In [10]:
# Define the deep neural network model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1=10
hidden_nodes_layer2=5

nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 22857 samples
Epoch 1/50
22857/22857 [==============================] - 3s 150us/sample - loss: 0.4258 - accuracy: 0.8382
Epoch 2/50
22857/22857 [==============================] - 2s 109us/sample - loss: 0.3751 - accuracy: 0.8705
Epoch 3/50
22857/22857 [==============================] - 4s 158us/sample - loss: 0.3723 - accuracy: 0.8725
Epoch 4/50
22857/22857 [==============================] - 6s 248us/sample - loss: 0.3709 - accuracy: 0.8726
Epoch 5/50
22857/22857 [==============================] - 4s 183us/sample - loss: 0.3702 - accuracy: 0.8726
Epoch 6/50
22857/22857 [==============================] - 3s 149us/sample - loss: 0.3695 - accuracy: 0.8726
Epoch 7/50
22857/22857 [==============================] - 4s 178us/sample - loss: 0.3692 - accuracy: 0.8726
Epoch 8/50
22857/22857 [==============================] - 3s 140us/sample - loss: 0.3686 - accuracy: 0.8726 - loss:
Epoch 9/50
22857/22857 [==============================] - 3s 136us/sample - loss: 0.3686 - accuracy: 0.87